# Ejercicio Flask API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/v2/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/v2/ingest_data)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/v2/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con Flask.

**Entregable**: repositorio github.

In [41]:
import pandas as pd
import sqlite3

In [42]:
df = pd.read_csv('data/Advertising.csv', index_col=0)
df.head()

,TV,radio,newpaper,sales
0,230.1,37.8,6s9.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0


In [43]:
df['newpaper'] = df['newpaper'].str.replace('s',"").astype(float)
df.rename(columns={'newpaper':'newspaper'}, inplace=True)
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


In [45]:
connection = sqlite3.connect('data/advertising.db')
# cursor = connection.cursor()
# cursor.execute("SELECT * FROM campañas").fetchall()

In [ ]:
# connection.close()

In [46]:
df.to_sql(name="campañas", con=connection, index=False, if_exists='replace')

200

In [20]:
import pickle

In [14]:
with open('data/advertising_model', 'rb') as file:
    model = pickle.load(file)

c:\Users\Miguel Angel\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
model

Ridge(alpha=5500, tol=0.001)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

In [16]:
# results_cv = cross_val_score(model, df.iloc[:,:-1], df.iloc[:,-1], cv=10, scoring="neg_mean_absolute_error")
# print(results_cv)
# print(results_cv.mean())
# print(results_cv.std())

In [21]:
def train_model(df):

    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]

    model = RandomForestRegressor()
    params = {"max_depth":[3,5],
              "max_features": [1,3]}
    
    gs = GridSearchCV(model , param_grid=params, cv=5, scoring="neg_mean_absolute_error")
    results = gs.fit(X, y)
    with open('data/new_advertising_model.pkl', 'wb') as file:
        pickle.dump(results.best_estimator_, file)
    

In [22]:
train_model(df)

In [28]:
with open('data/new_advertising_model.pkl', 'rb') as file:
    new_model = pickle.load(file)

In [24]:
results_cv = cross_val_score(new_model, df.iloc[:,:-1], df.iloc[:,-1], cv=10, scoring="neg_mean_absolute_error")
print(results_cv)
print(results_cv.mean())
print(results_cv.std())

[-21.21843146 -13.26110954 -14.32748476 -14.4083129  -15.73539291
 -17.54310854 -22.87981129 -12.91029388 -15.28279616 -17.6065132 ]
-16.51732546384958
3.159313545297048


In [47]:
connection = sqlite3.connect('data/advertising.db')
cursor = connection.cursor()
query = '''SELECT * FROM campañas'''
result = cursor.execute(query).fetchall()

In [48]:
pd.DataFrame(result)

,0,1,2,3
0,230.1,37.8,69.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


In [51]:
new_model.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestRegressor(max_depth=3, max_features=1)